In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%shell
# install MTCNN
pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.5 MB/s eta 0:00:00


In [ ]:
%%shell
# Run with parallelism
python drive/MyDrive/mad/tests/crop_dataset_builder.py \
  --input_path drive/MyDrive/mad/datasets/frll-morphs/ \
  --output_path output \
  --multiprocessing --workers 12

2025-08-31 17:15:16.033579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-31 17:15:16.051551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756660516.072662    1861 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756660516.079083    1861 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756660516.095701    1861 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
from pathlib import Path
from zipfile import ZipFile, ZIP_DEFLATED
from tqdm import tqdm

# --- Paths ---
src = Path("output")  # folder to zip
zip_local = Path("/content/output.zip")  # temporary zip in Colab local storage
zip_drive = Path("/content/drive/MyDrive/mad/tests/output_crop.zip")  # final zip on Drive

# --- 1. Zip with progress bar ---
files = [f for f in src.rglob("*") if f.is_file()]

print("Zipping files...")
with ZipFile(zip_local, "w", compression=ZIP_DEFLATED) as zipf:
    for f in tqdm(files, desc="Zipping", unit="file"):
        rel = f.relative_to(src)
        zipf.write(f, rel)

# --- 2. Copy zip to Drive with progress bar ---
print("Copying zip to Drive...")
buffer_size = 1024 * 1024  # 1 MB chunks
total_size = zip_local.stat().st_size

with open(zip_local, "rb") as fsrc, open(zip_drive, "wb") as fdst:
    with tqdm(total=total_size, unit="B", unit_scale=True, desc="Copying") as pbar:
        while True:
            buf = fsrc.read(buffer_size)
            if not buf:
                break
            fdst.write(buf)
            pbar.update(len(buf))

print(f"\n✅ Zip saved at: {zip_drive}")

Zipping files...


Zipping: 100%|██████████| 7266/7266 [00:35<00:00, 206.67file/s]


Copying zip to Drive...


Copying: 100%|██████████| 943M/943M [00:01<00:00, 629MB/s]


✅ Zip saved at: /content/drive/MyDrive/mad/tests/output_crop.zip


In [ ]:
import time
from google.colab import drive, runtime

drive.flush_and_unmount()

runtime.unassign()